In [ ]:
import os
import subprocess
import csv
from datetime import datetime

import matplotlib.pyplot as plt  # MANHATTAN

# === Konfiguration ===
REPO_PATH = "/home/pi/Robo_challenge"  # Pfad zum lokalen Git-Repository
SUBMISSION_DIR = os.path.join(REPO_PATH, "submissions")
TEAM_NAME = "Zumi123"  # <-- Teamnamen hier anpassen

# === Hilfsfunktionen ===
def get_unique_filename():
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    return "{}_result_{}.csv".format(TEAM_NAME, timestamp)

def create_submission_file():
    """Erzeugt eine neue CSV-Datei mit Kopfzeile für Logs."""
    if not os.path.exists(SUBMISSION_DIR):
        os.makedirs(SUBMISSION_DIR)

    file_name = get_unique_filename()
    file_path = os.path.join(SUBMISSION_DIR, file_name)

    with open(file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Zeit", "Aktion"])

    print("📄 Neue Logdatei erstellt:", file_path)
    return file_path

def log_aktion(filepath, aktion="Aktion nicht angegeben"):
    """Fügt eine neue Zeile mit Zeitstempel und Aktion hinzu."""
    now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open(filepath, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([now, aktion])
    print("📝 Aktion geloggt:", aktion)

def upload_submission(file_path):
    """Fügt Datei zu Git hinzu, commitet und pusht sie."""
    try:
        COMMIT_MESSAGE = "Submission by {} - {}".format(TEAM_NAME, datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
        subprocess.run(["git", "-C", REPO_PATH, "add", file_path], check=True)
        subprocess.run(["git", "-C", REPO_PATH, "commit", "-m", COMMIT_MESSAGE], check=True)
        subprocess.run(["git", "-C", REPO_PATH, "push", "origin", "main"], check=True)
        print("✅ Datei erfolgreich gepusht:", os.path.basename(file_path))
    except subprocess.CalledProcessError as e:
        print("❌ Git Error:", e)
    except Exception as e:
        print("❌ Allgemeiner Fehler:", e)

def upload_face_image(image_path):
    """Gespeichertes Gesicht als PNG in Git hochladen."""
    try:
        COMMIT_MESSAGE = "Gesicht erkannt von {} - {}".format(TEAM_NAME, datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
        subprocess.run(["git", "-C", REPO_PATH, "add", image_path], check=True)
        subprocess.run(["git", "-C", REPO_PATH, "commit", "-m", COMMIT_MESSAGE], check=True)
        subprocess.run(["git", "-C", REPO_PATH, "push", "origin", "main"], check=True)
        print("✅ Gesicht erfolgreich gepusht:", os.path.basename(image_path))
    except subprocess.CalledProcessError as e:
        print("❌ Git Error beim Bild-Upload:", e)
    except Exception as e:
        print("❌ Allgemeiner Fehler beim Bild-Upload:", e)

# === MANHATTAN distance code START ===
positions = [(0, 0)]
directions = ["north", "east", "south", "west"]
current_direction_idx = 0
manhattan_distances = [0]
manhattan_csv_file = os.path.join(SUBMISSION_DIR, f"{TEAM_NAME}_manhattan_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv")

def log_position(x, y):
    last_x, last_y = positions[-1]
    dist = abs(x - last_x) + abs(y - last_y)
    cumulative_dist = manhattan_distances[-1] + dist
    positions.append((x, y))
    manhattan_distances.append(cumulative_dist)
    header_needed = not os.path.exists(manhattan_csv_file)
    with open(manhattan_csv_file, "a", newline='') as f:
        writer = csv.writer(f)
        if header_needed:
            writer.writerow(["X", "Y", "Cumulative_Manhattan_Distance"])
        writer.writerow([x, y, cumulative_dist])

def plot_path_and_save():
    xs, ys = zip(*positions)
    plt.figure(figsize=(6, 6))
    plt.plot(xs, ys, marker="o")
    plt.title("Zumi Path (Grid)")
    plt.xlabel("X Position")
    plt.ylabel("Y Position")
    plt.grid(True)
    png_path = manhattan_csv_file.replace(".csv", ".png")
    plt.savefig(png_path)
    plt.close()
    return png_path

def update_position(move_type="forward", turn=None):
    global current_direction_idx
    x, y = positions[-1]
    if move_type == "turn":
        if turn == "left":
            current_direction_idx = (current_direction_idx - 1) % 4
        elif turn == "right":
            current_direction_idx = (current_direction_idx + 1) % 4
        return
    elif move_type == "forward":
        direction = directions[current_direction_idx]
        if direction == "north":
            y += 1
        elif direction == "south":
            y -= 1
        elif direction == "east":
            x += 1
        elif direction == "west":
            x -= 1
        log_position(x, y)

In [ ]:
from zumi.zumi import Zumi
import time

from zumi.util.screen import Screen

from zumi.util.camera import Camera
from zumi.util.vision import Vision

from zumi.personality import Personality

from zumi.protocol import Note

import cv2

# === Zumi Setup and Globals ===
zumi = Zumi()
zumi.reset_gyro()
camera = Camera()
screen = Screen()
personality = Personality(zumi, screen)
vision = Vision()

screen.happy()

#GLOBAL VARs
whiteTreshold = 150
circRounds = 1
speed = 15
jump_back_threshold = 20

logfile = create_submission_file()
face_count = 0
object_count = 0
qr_count = 0
image_path = ""

def corr_line(heading):
    head = heading
    ir_readings = zumi.get_all_IR_data()
    
    left_ir_front = ir_readings[5]
    right_ir_front = ir_readings[0]
    front_threshold = 50

    if left_ir_front < front_threshold or right_ir_front < front_threshold:
        print("Hindernis im corr_line erkannt – Objekt voraus!")
        zumi.stop()
        object_ahead(zumi, screen, personality)
        return head

    left_ir_bottom = ir_readings[3]
    right_ir_bottom = ir_readings[1]

    if left_ir_bottom > whiteTreshold and right_ir_bottom <= whiteTreshold:
        head += 5
    elif right_ir_bottom > whiteTreshold and left_ir_bottom <= whiteTreshold:
        head -= 5
    elif left_ir_bottom <= whiteTreshold and right_ir_bottom <= whiteTreshold:
        zumi.stop()
        head = "weiss"

    return head

def smooth(raw_values):
    smoothed = []
    for i in range(len(raw_values)):
        if i < 2:
            window = raw_values[0:i+1]
        else:
            window = raw_values[i-2:i+1]
        avg = sum(window) / len(window)
        smoothed.append(avg)
    return smoothed

def search_curve(zumi):
    zumi.stop()
    print("Linie verloren – starte gezielte Suche...")

    zumi.reset_gyro()
    time.sleep(0.1)

    zumi.signal_left_on()
    zumi.turn(90)
    update_position(move_type="turn", turn="left") # MANHATTAN
    zumi.signal_left_off()
    time.sleep(0.3)
    ir_readings = zumi.get_all_IR_data()
    right_ir_bottom = ir_readings[1]
    left_ir_bottom = ir_readings[3]
    

    if left_ir_bottom > whiteTreshold and right_ir_bottom > whiteTreshold:
        print("Linie im Westen gefunden.")
        zumi.reset_gyro()
        log_aktion(logfile, aktion="Linkskurve Zeile 95")
        return 0
    if left_ir_bottom > whiteTreshold:
        print("Linker Sensor sieht Linie – feinjustiere weiter nach links...")
        zumi.turn_left(20)
        update_position(move_type="turn", turn="left") # MANHATTAN
        time.sleep(0.3)
        ir_readings = zumi.get_all_IR_data()
        if ir_readings[1] > whiteTreshold and ir_readings[3] > whiteTreshold:
            print("Linie nach Feinjustierung im westen gefunden.")
            zumi.reset_gyro()
            return 0

    zumi.signal_right_on()
    zumi.turn(-90)
    update_position(move_type="turn", turn="right") # MANHATTAN
    time.sleep(0.3)
    zumi.signal_right_off()
    ir_readings = zumi.get_all_IR_data()
    right_ir_bottom = ir_readings[1]
    left_ir_bottom = ir_readings[3]

    if left_ir_bottom > whiteTreshold and right_ir_bottom > whiteTreshold:
        print("Linie im Osten gefunden.")
        zumi.reset_gyro()
        log_aktion(logfile, aktion="Rechts kurve Zeile 121")
        return 0
    if right_ir_bottom > whiteTreshold:
        print("Rechter Sensor sieht Linie – feinjustiere weiter nach rechts...")
        zumi.turn_right(20)
        update_position(move_type="turn", turn="right") # MANHATTAN
        time.sleep(0.3)
        ir_readings = zumi.get_all_IR_data()
        if ir_readings[1] > whiteTreshold and ir_readings[3] > whiteTreshold:
            print("Linie nach Feinjustierung im Osten gefunden.")
            zumi.reset_gyro()
            return 0

    print("Keine Linie erkannt.")
    return None

def follow_circle(zumi, direction="right"):
    global object_count
    global qr_count
    global face_count
    camera.close()
    heading = 0
    turn_count = 0
    object_count_ = object_count - qr_count - qr_count
    circRounds = object_count_ - face_count
    max_turns = 4 * circRounds
    screen.draw_text_center("Kreisverkehrfahren")
    time.sleep(0.5)
    screen.draw_text_center(str(circRounds))
    rev = "right"
    log_aktion(logfile, aktion="Starte Kreisverkehr Zeile 152")
    while True:
        heading = corr_line(heading)
        if heading == "weiss":
            print("Linie verloren – Turn QR Circle richtung...")

            if direction == "right":
                zumi.signal_right_on()
                zumi.turn(-90)
                update_position(move_type="turn", turn="right") # MANHATTAN
                zumi.signal_right_off()
                time.sleep(0.3)
                zumi.reset_gyro()
                new_heading = 0
                rev = "left"

            elif direction == "left":
                zumi.signal_left_on()
                zumi.turn(90)
                update_position(move_type="turn", turn="left") # MANHATTAN
                zumi.signal_left_off()
                time.sleep(0.3)
                zumi.reset_gyro()
                new_heading = 0
                rev = "right"

            turn_count += 1
            print("Abbiegevorgang:", turn_count, "von", max_turns)
            screen.draw_text_center("Abbiegevorgang: {} von {}".format(turn_count, max_turns))

            if new_heading is not None:
                heading = new_heading
                if turn_count >= max_turns:
                    while True:
                        print("LOOP AFTER CIRCLE")
                        heading = corr_line(heading)
                        if heading == "weiss":
                            print("Weiße Fläche erkannt")
                            log_aktion(logfile, aktion = "Kreisverkehr verlassen Zeile 189")
                            if rev == "left":
                                zumi.signal_left_on()
                                zumi.turn(90)
                                update_position(move_type="turn", turn="left") # MANHATTAN
                                zumi.signal_left_off()
                                zumi.reset_gyro()
                            elif rev == "right":
                                zumi.signal_right_on()
                                zumi.turn(-90)
                                update_position(move_type="turn", turn="right") # MANHATTAN
                                zumi.signal_right_off()
                                zumi.reset_gyro()
                            return 0
                        zumi.go_straight(10, heading)
                        update_position(move_type="forward") # MANHATTAN
                        time.sleep(0.005)
                continue
            else:
                print("Suche gescheitert – Programmabbruch.")
                break

        print("HEADING CIRCLE FOLLOW END: ", heading)
        zumi.go_straight(speed, heading)
        update_position(move_type="forward") # MANHATTAN
        time.sleep(0.02)
    zumi.stop()

def follow_line(zumi, abs_ir_threshold=50):
    heading = 0
    start_time = time.time()
    print("HEADING START: ", heading)

    ir_readings = zumi.get_all_IR_data()
    right_ir_history = [ir_readings[0]] * 10
    left_ir_history = [ir_readings[5]] * 10

    while True:
        ir_readings = zumi.get_all_IR_data()
        left_ir_front = ir_readings[5]
        right_ir_front = ir_readings[0]

        right_ir_history.append(right_ir_front)
        left_ir_history.append(left_ir_front)
        if len(right_ir_history) > 10:
            print(right_ir_history)
            right_ir_history.pop(0)
        if len(left_ir_history) > 10:
            print(left_ir_history)
            left_ir_history.pop(0)

        heading = corr_line(heading)
        if heading == "weiss":
            print("Linie verloren – suche neue Richtung...")
            new_heading = search_curve(zumi)
            if new_heading is not None:
                heading = new_heading
                continue
            else:
                print("Suche gescheitert – Programmabbruch.")
                break

        print("HEADING FOLLOW END: ", heading)
        zumi.go_straight(speed, heading)
        update_position(move_type="forward") # MANHATTAN
        time.sleep(0.02)

    zumi.stop()

# (Your remaining functions and helpers remain unchanged, exactly as before!)

def drive_course(zumi, total_time=60):
    global logfile
    if logfile is None:
        logfile = create_submission_file()

    print("Starte Parkour-Fahrt")
    screen.draw_text_center("Starte Parkour-Fahrt")
    start_time = time.time()
    log_aktion(logfile, aktion="Start der Fahrt Zeile 608")

    while time.time() - start_time < total_time:
        print("Starte neuen Block...")
        follow_line(zumi)

    zumi.stop()
    end_time = time.time()
    log_aktion(logfile, aktion="End der Fahrt")

    komp_time = end_time - start_time
    log_aktion(logfile, aktion="Total time = " + str(komp_time))
    print("Parkour abgeschlossen.")


In [ ]:
# Zeitbegrenzung noch auf alt 120 sec = 2 mins, -> neu = 6 mins -> 360
drive_course(zumi, total_time=360)

In [ ]:
zumi.stop()

In [ ]:
camera.close()

In [ ]:
upload_submission(logfile)  # Upload the latest submission
if image_path:
    upload_face_image(image_path)
# === MANHATTAN ===
png_path = plot_path_and_save()
upload_submission(manhattan_csv_file)
upload_submission(png_path)